# Create masks of brain regions 
- We create masks for 180 brain regions according to [MNI_Glasser_HCP_2019_v1.0](https://afni.nimh.nih.gov/pub/dist/atlases/MNI_HCP/MNI_Glasser_HCP_2019_v1.0/)

## 1. remove motion nuisance and demean brick data

In [ ]:
%%bash 
#!/bin/bash
export PATH=$PATH:/home/statespace/abin

# remove nuisance from afni using 3dTproject
project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/igeo_process"
fMRI_data_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_data/spatialfMRI_nii"

sub_list="sub-01 sub-02 sub-03 sub-04 sub-05 sub-06 sub-07 sub-08 sub-09 sub-10 sub-11 sub-12 sub-13 sub-14 sub-15 sub-16 sub-17 sub-18 sub-19 sub-20"

for sub in ${sub_list}
do
    
    subject=${sub}

    preprocessed_dir=${project_dir}/preprocess/preprocessed/afni_2023/${subject}
    output_dir=${project_dir}/process/processed/${subject}

    mkdir -p ${output_dir}/shared_regions

    output_current_dir=${output_dir}/shared_regions

    run_list="01 02 03 04 05 06 07 08 09 10 11 12 13 14"
    for run in ${run_list}
    do

        # write motion file
        1d_tool.py -infile ${preprocessed_dir}/dfile.r${run}.1D                        \
            -demean -overwrite -write ${output_current_dir}/motion_r${run}.1D

        # remove motion noise
        3dTproject -polort 4 \
            -input ${preprocessed_dir}/pb04.${subject}.r${run}.combine+orig.           \
            -prefix ${output_current_dir}/pb04.${subject}.r${run}.combine+orig.        \
            -ort ${output_current_dir}/motion_r${run}.1D                               \
            -overwrite 

        # mean all TRs
        3dTstat -mean -prefix ${output_current_dir}/pb04.${subject}.r${run}.mean       \
            ${output_current_dir}/pb04.${subject}.r${run}.combine+orig.

        # demean
        3dcalc -a ${output_current_dir}/pb04.${subject}.r${run}.combine+orig.          \
            -b ${output_current_dir}/pb04.${subject}.r${run}.mean+orig                 \
            -expr 'a-b'                                                                \
            -prefix ${output_current_dir}/pb04.${subject}.r${run}.combine+demean   

        # afni to nifti
        3dcalc -a ${output_current_dir}/pb04.${subject}.r${run}.combine+demean+orig.   \
            -expr 'a'                                                                  \
            -prefix ${output_current_dir}/pb04.${subject}.r${run}.combine+demean.nii.gz        
    done
done

## 2. concatenate all runs

In [ ]:
%%bash
export PATH=$PATH:/home/statespace/abin

# concatenate all runs

project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/igeo_process"
fMRI_data_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_data/spatialfMRI_nii"

sub_list="sub-01 sub-02 sub-03 sub-04 sub-05 sub-06 sub-07 sub-08 sub-09 sub-10 sub-11 sub-12 sub-13 sub-14 sub-15 sub-16 sub-17 sub-18 sub-19 sub-20"

for sub in ${sub_list}
do

        subject=${sub}

        preprocessed_dir=${project_dir}/preprocess/preprocessed/afni_2023/${subject}
        output_dir=${project_dir}/process/processed/${subject}

        output_current_dir=${output_dir}/shared_regions

        cd ${output_current_dir}

        3dTcat -DAFNI_GLOB_SELECTORS=YES -relabel -prefix ${output_current_dir}/pb04.${subject}.rall.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r01.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r02.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r03.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r04.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r05.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r06.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r07.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r08.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r09.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r10.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r11.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r12.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r13.combine+demean+orig \
                ${output_current_dir}/pb04.${subject}.r14.combine+demean+orig 
done

## 3. create masks of brain regions

In [ ]:
%%bash 
export PATH=$PATH:/home/statespace/abin

# create mask
project_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_analysis/igeo_process"
fMRI_data_dir="/media/statespace/Spatial/sptialworkspace/spatialfMRI/fMRI_data/spatialfMRI_nii"

preprocessed_dir=${project_dir}/preprocess/preprocessed

sub_list="sub-01 sub-02 sub-03 sub-04 sub-05 sub-06 sub-07 sub-08 sub-09 sub-10 sub-11 sub-12 sub-13 sub-14 sub-15 sub-16 sub-17 sub-18 sub-19 sub-20"

for sub in ${sub_list}
do
	echo ${sub}

	cd ${preprocessed_dir}/afni_2023/${sub}
    
    output_dir=${project_dir}/process/processed/${sub}

    mkdir -p ${output_dir}/shared_regions

    output_current_dir=${output_dir}/shared_regions

    # run_list="01 02 03 04 05 06 07 08 09 10 11 12 13 14"
    run_list="01"
    for run in ${run_list}
    do
        3dNwarpApply -iwarp -nwarp './anat.un.aff.qw_WARP.nii anat.un.aff.Xat.1D' \
            -master pb04.${sub}.r${run}.combine+orig.  \
            -source /home/statespace/abin/MNI_Glasser_HCP_v1.0.nii.gz \
            -prefix ${output_current_dir}/${sub}_MNI_Glasser_HCP_neural_mask.nii.gz \
            -interp NN -overwrite            
    done
done